In [2]:
import sys
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from sklearn import preprocessing
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [3]:
my_libs_dir = '../'
if not my_libs_dir in sys.path:
    sys.path.append(my_libs_dir)  # add the path to my_lib directory 

# The following lines are needed to auto-reload my library file
# Without these lines, my library file is read only once and
# modifications of my library file are not reflected.
%load_ext autoreload
%autoreload 1
%aimport my_libs.linear_reg
# import from my library file
from my_libs.linear_reg import step_aic_forward, calc_vifs

In [4]:
%config InlineBackend.figure_formats = {'png', 'retina'} #high-reso images
font = {'family' : 'Yu Mincho'}
matplotlib.rc('font', **font)

# To show all rows and columns in the results 
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [5]:
csv_in = 'winequality-red_modified-utf8.txt'
df_all = pd.read_csv(csv_in, delimiter='\s+', skiprows=13, header=0)
# no header in csv, so set columns explicitly
# df_all.columns = ['sex', 'len', 'd', 'h', 'w_all', 'w_meat', 'w_gut', 'w_shell', 'ring']
print(df_all.shape)
print(df_all.info())
display(df_all.head())

(1599, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed_acidity         1599 non-null   float64
 1   volatile_acidity      1599 non-null   float64
 2   citric_acid           1599 non-null   float64
 3   residual_sugar        1599 non-null   float64
 4   chlorides             1598 non-null   float64
 5   free_sulfur_dioxide   1599 non-null   float64
 6   total_sulfur_dioxide  1599 non-null   float64
 7   density               1597 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB
None


fixed_acidity  volatile_acidity  citric_acid  residual_sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9        NaN   

   free_sulfur_dioxide  total_sulfur_dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0      NaN  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0      NaN  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2      9.8        5  
3      9.8        6  
4      9.4        5

In [6]:
df_all = df_all.dropna().reset_index(drop=True)
print(df_all.shape)
display(df_all.head())

(1596, 12)


fixed_acidity  volatile_acidity  citric_acid  residual_sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.76         0.04             2.3      0.092   
2            7.4              0.66         0.00             1.8      0.075   
3            7.9              0.60         0.06             1.6      0.069   
4            7.3              0.65         0.00             1.2      0.065   

   free_sulfur_dioxide  total_sulfur_dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 15.0                  54.0   0.9970  3.26       0.65   
2                 13.0                  40.0   0.9978  3.51       0.56   
3                 15.0                  59.0   0.9964  3.30       0.46   
4                 15.0                  21.0   0.9946  3.39       0.47   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2      9.4        5  
3      9.4        5  
4     10.0        7

In [7]:
X_all = df_all.loc[:, 'fixed_acidity':'alcohol']  # explanatory variables
y = df_all['quality']  # objective variable
print('X_all:', X_all.shape)
print('y:', y.shape)

X_all: (1596, 11)
y: (1596,)


In [8]:
corr_all = X_all.corr(method='pearson')
display(corr_all)

fixed_acidity  volatile_acidity  citric_acid  \
fixed_acidity              1.000000         -0.254796     0.671376   
volatile_acidity          -0.254796          1.000000    -0.550864   
citric_acid                0.671376         -0.550864     1.000000   
residual_sugar             0.115231          0.001749     0.143933   
chlorides                  0.094037          0.061071     0.204452   
free_sulfur_dioxide       -0.154094         -0.011231    -0.060859   
total_sulfur_dioxide      -0.113752          0.076463     0.035440   
density                    0.668356          0.022282     0.365623   
pH                        -0.683007          0.234924    -0.542063   
sulphates                  0.183527         -0.261792     0.313457   
alcohol                   -0.061580         -0.202022     0.109363   

                      residual_sugar  chlorides  free_sulfur_dioxide  \
fixed_acidity               0.115231   0.094037            -0.154094   
volatile_acidity            0.001749   0.061071            -0.011231   
citric_acid                 0.143933   0.204452            -0.060859   
residual_sugar              1.000000   0.055469             0.187006   
chlorides                   0.055469   1.000000             0.005389   
free_sulfur_dioxide         0.187006   0.005389             1.000000   
total_sulfur_dioxide        0.203091   0.047337             0.667541   
density                     0.355759   0.200882            -0.021855   
pH                         -0.085640  -0.265166             0.071304   
sulphates                   0.005230   0.371164             0.051475   
alcohol                     0.041679  -0.221426            -0.069386   

                      total_sulfur_dioxide   density        pH  sulphates  \
fixed_acidity                    -0.113752  0.668356 -0.683007   0.183527   
volatile_acidity                  0.076463  0.022282  0.234924  -0.261792   
citric_acid                       0.035440  0.365623 -0.542063   0.313457   
residual_sugar                    0.203091  0.355759 -0.085640   0.005230   
chlorides                         0.047337  0.200882 -0.265166   0.371164   
free_sulfur_dioxide               0.667541 -0.021855  0.071304   0.051475   
total_sulfur_dioxide              1.000000  0.071252 -0.065735   0.042895   
density                           0.071252  1.000000 -0.342146   0.148960   
pH                               -0.065735 -0.342146  1.000000  -0.196633   
sulphates                         0.042895  0.148960 -0.196633   1.000000   
alcohol                          -0.205651 -0.495957  0.206091   0.093188   

                       alcohol  
fixed_acidity        -0.061580  
volatile_acidity     -0.202022  
citric_acid           0.109363  
residual_sugar        0.041679  
chlorides            -0.221426  
free_sulfur_dioxide  -0.069386  
total_sulfur_dioxide -0.205651  
density              -0.495957  
pH                    0.206091  
sulphates             0.093188  
alcohol               1.000000

In [9]:
plt.figure(figsize=(8,8))
sns.heatmap(corr_all,annot=True,fmt='.2f',cmap='BrBG_r')
plt.show()

findfont: Font family ['Yu Mincho'] not found. Falling back to DejaVu Sans.


In [10]:
X_all_c = sm.add_constant(X_all)
model = sm.OLS(y, X_all_c)
results = model.fit()
print('R2:', results.rsquared)
print('Adj R2:', results.rsquared_adj)
print('p-values (F-statistic)', results.f_pvalue)
print(results.summary())

R2: 0.36008321392086784
Adj R2: 0.3556393473508739
p-values (F-statistic) 6.168748635716057e-145
                            OLS Regression Results                            
Dep. Variable:                quality   R-squared:                       0.360
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     81.03
Date:                Sun, 02 May 2021   Prob (F-statistic):          6.17e-145
Time:                        17:32:04   Log-Likelihood:                -1567.6
No. Observations:                1596   AIC:                             3159.
Df Residuals:                    1584   BIC:                             3224.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
------------------------

In [11]:
X_scaled = preprocessing.scale(X_all)
y_scaled = preprocessing.scale(y)
model = sm.OLS(y_scaled, X_scaled)
results = model.fit()
print('R2:', results.rsquared)
print('Adj R2:', results.rsquared_adj)
print('p-values (F-statistic)', results.f_pvalue)
print(results.summary())

R2: 0.36008321392086784
Adj R2: 0.3556421510521799
p-values (F-statistic) 4.948632336168239e-145
                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.360
Model:                            OLS   Adj. R-squared (uncentered):              0.356
Method:                 Least Squares   F-statistic:                              81.08
Date:                Sun, 02 May 2021   Prob (F-statistic):                   4.95e-145
Time:                        17:32:05   Log-Likelihood:                         -1908.4
No. Observations:                1596   AIC:                                      3839.
Df Residuals:                    1585   BIC:                                      3898.
Df Model:                          11                                                  
Covariance Type:            nonrobust                                                  
                 coef  

In [12]:
# NOTE: make DataFrames corresponding to X_scaled and y_scaled.
dfX_scaled = pd.DataFrame(X_scaled, columns=X_all.columns)
dfy_scaled = pd.Series(y_scaled, name=y.name)
exog = list(dfX_scaled.columns)  # Initial set = all explanatory variables
endog = [dfy_scaled.name]  # Objective variables
df_scaled = pd.concat([dfX_scaled, dfy_scaled], axis=1)
results_aic=step_aic_forward(smf.ols, exog, endog, data=df_scaled)

AIC: 4531.252, formula: quality ~ 1
AIC: 4506.508, formula: quality ~ chlorides
AIC: 4270.337, formula: quality ~ volatile_acidity
AIC: 4532.954, formula: quality ~ residual_sugar
AIC: 4483.646, formula: quality ~ density
AIC: 4529.172, formula: quality ~ free_sulfur_dioxide
AIC: 4508.780, formula: quality ~ fixed_acidity
AIC: 4428.999, formula: quality ~ sulphates
AIC: 4477.538, formula: quality ~ total_sulfur_dioxide
AIC: 4450.253, formula: quality ~ citric_acid
AIC: 4528.011, formula: quality ~ pH
AIC: 4123.148, formula: quality ~ alcohol
AIC: 4123.949, formula: quality ~ alcohol + chlorides
AIC: 3928.074, formula: quality ~ alcohol + volatile_acidity
AIC: 4125.070, formula: quality ~ alcohol + residual_sugar
AIC: 4114.882, formula: quality ~ alcohol + density
AIC: 4124.514, formula: quality ~ alcohol + free_sulfur_dioxide
AIC: 4076.123, formula: quality ~ alcohol + fixed_acidity
AIC: 4033.266, formula: quality ~ alcohol + sulphates
AIC: 4108.632, formula: quality ~ alcohol + total_

In [13]:
print(results_aic.aic)
print(results_aic.model.exog_names)
print(results_aic.model.endog_names)

3835.4763012244853
['Intercept', 'alcohol', 'volatile_acidity', 'sulphates', 'total_sulfur_dioxide', 'chlorides', 'pH', 'free_sulfur_dioxide']
quality


In [14]:
endogs = results_aic.model.endog_names
exogs = results_aic.model.exog_names.copy()
exogs.remove('Intercept')
#print(exogs)  # debug
X_c = sm.add_constant(X_all[exogs])
vifs = calc_vifs(X_c, y)
display(vifs)

VIF
const                 618.539992
alcohol                 1.220248
volatile_acidity        1.242579
sulphates               1.322258
total_sulfur_dioxide    1.943518
chlorides               1.333407
pH                      1.255038
free_sulfur_dioxide     1.882888

In [15]:
X_final_scaled = dfX_scaled[exogs]
model_final_scaled = sm.OLS(y_scaled, X_final_scaled)
results_final_scaled = model_final_scaled.fit()
print(results_final_scaled.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.359
Model:                            OLS   Adj. R-squared (uncentered):              0.356
Method:                 Least Squares   F-statistic:                              127.1
Date:                Sun, 02 May 2021   Prob (F-statistic):                   1.48e-148
Time:                        17:32:07   Log-Likelihood:                         -1909.7
No. Observations:                1596   AIC:                                      3833.
Df Residuals:                    1589   BIC:                                      3871.
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
                           coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------

In [16]:
print(results_final_scaled.params)

alcohol                 0.381874
volatile_acidity       -0.223534
sulphates               0.185601
total_sulfur_dioxide   -0.142075
chlorides              -0.117664
pH                     -0.092192
free_sulfur_dioxide     0.065957
dtype: float64


In [17]:
# Step 10. Stat. test for MLR equation 
print('p-values (F-statistic)', results_final_scaled.f_pvalue)

p-values (F-statistic) 1.4839271358964914e-148


In [19]:
X_final_c = sm.add_constant(X_all[exogs])
model_final = sm.OLS(y, X_final_c)
results_final = model_final.fit()
print('R2:', results_final.rsquared)
print('Adj R2:', results_final.rsquared_adj)
print(results_final.summary())

R2: 0.3589972226304371
Adj R2: 0.3561716436369945
                            OLS Regression Results                            
Dep. Variable:                quality   R-squared:                       0.359
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     127.1
Date:                Sun, 02 May 2021   Prob (F-statistic):          1.85e-148
Time:                        17:36:55   Log-Likelihood:                -1568.9
No. Observations:                1596   AIC:                             3154.
Df Residuals:                    1588   BIC:                             3197.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------

In [20]:
print(results_final.params)

const                   4.426027
alcohol                 0.289401
volatile_acidity       -1.009819
sulphates               0.884003
total_sulfur_dioxide   -0.003487
chlorides              -2.018132
pH                     -0.482495
free_sulfur_dioxide     0.005091
dtype: float64
